# Movie Recommadation Engine

In [1]:
# Importing all necessry libraries
import pandas as pd
import numpy as np
import ast
from ast import literal_eval
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
# reading the datasets
df_movies = pd.read_csv('/content/tmdb_5000_movies.csv')
df_credits = pd.read_csv('/content/tmdb_5000_credits.csv')

In [3]:
# checking the sample row for movies
df_movies.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
698,70000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""...",NaN,9837,"[{""id"": 488, ""name"": ""moses""}, {""id"": 1160, ""n...",en,The Prince of Egypt,This is the extraordinary tale of two brothers...,29.464852,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1998-12-15,218613188,99.0,"[{""iso_639_1"": ""he"", ""name"": ""\u05e2\u05b4\u05...",Released,The Power Is Real.,The Prince of Egypt,6.8,817


In [4]:
# checking the sample row for credits
df_credits.sample()

,movie_id,title,cast,crew
3377,104896,A Dog Of Flanders,"[{""cast_id"": 3, ""character"": ""Jehan"", ""credit_...","[{""credit_id"": ""52fe4a3fc3a36847f81c3501"", ""de..."


In [5]:
# checking shape for movies
df_movies.shape

(4803, 20)

In [6]:
# checking shap for credits
df_credits.shape

(4803, 4)

In [7]:
# merging movies data set with credits dataset
movies = df_movies.merge(df_credits,on='title')
movies.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
1788,15000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,951,"[{""id"": 853, ""name"": ""crime fighter""}, {""id"": ...",en,Kindergarten Cop,Hard-edged cop John Kimble gets more than he b...,5.411154,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...",...,111.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Go ahead, you tell him you didn't do your home...",Kindergarten Cop,5.8,628,951,"[{""cast_id"": 1, ""character"": ""Detective John K...","[{""credit_id"": ""52fe4292c3a36847f8029389"", ""de..."


In [8]:
# Lets drop following columns (not needed)
# budget, homepage, id, original_language, original_title, popularity, production_comapny, production_countries, release-date
# new data set after droping some columns
movies = movies[['movie_id','title','overview','genres','cast','crew','keywords']]
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
4163,312793,The Walking Deceased,THE WALKING DECEASED is the Scary Movie of the...,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...","[{""cast_id"": 1, ""character"": ""Green Bay"", ""cre...","[{""credit_id"": ""54f5fff9c3a3683a65000252"", ""de...","[{""id"": 11931, ""name"": ""spoof""}, {""id"": 12377,..."


In [9]:
# understanding columns before cleaning them
movies.iloc[0].overview
movies.iloc[0].genres
movies.iloc[0].cast
movies.iloc[0].crew
movies.iloc[0].keywords


'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [10]:
# understanding stats for the dataset
movies.describe()

,movie_id
count,4809.000000
mean,57120.571429
std,88653.369849
min,5.000000
25%,9012.000000
50%,14624.000000
75%,58595.000000
max,459488.000000


In [11]:
# checking the datatype
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   cast      4809 non-null   object
 5   crew      4809 non-null   object
 6   keywords  4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [12]:
movies.duplicated().sum()

0

In [13]:
# dropping null
movies.dropna(inplace=True)

Ast libray (literal_eval) change from '["user1", "user2" ... ]'  to  ["user1", "user2" ... ]

basically it converts '[]' string values to list values []

In [14]:
# keeping names only function
def filter(text):
    List = []
    for names in ast.literal_eval(text):
        List.append(names['name'])
    return List

In [15]:
# applying filter function
movies['genres'] = movies['genres'].apply(filter)
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
3114,18191,Edmond,Seemingly mild-mannered businessman Edmond Bur...,"[Drama, Thriller]","[{""cast_id"": 1, ""character"": ""Edmond"", ""credit...","[{""credit_id"": ""52fe47669251416c75098013"", ""de...","[{""id"": 242, ""name"": ""new york""}, {""id"": 366, ..."


In [16]:
# applying filter function
movies['keywords'] = movies['keywords'].apply(filter)
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
3826,34769,Defendor,"A crooked cop, a mob boss and the young girl t...","[Drama, Action, Comedy, Crime]","[{""cast_id"": 1, ""character"": ""Arthur Poppingto...","[{""credit_id"": ""5329b1b1c3a3683d74001a2c"", ""de...","[crime fighter, delusion, superhero]"


In [17]:
# applying filter function
movies['cast'] = movies['cast'].apply(filter)
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
3696,8981,Dear Frankie,Nine-year-old Frankie and his single mum Lizzi...,"[Drama, Family]","[Gerard Butler, Emily Mortimer, Jack McElhone,...","[{""credit_id"": ""57abaf1f9251416bf1000e6b"", ""de...","[scotland, father son relationship, single par..."


In [18]:
# keeping only 10 actors (cast) per movie
movies['cast'] = movies['cast'].apply(lambda x:x[0:10])

In [19]:
# keeping directors only function
def get_directors_names(text):
    directors_list = []

    jobs_data = ast.literal_eval(text)
    for person in jobs_data:
        if person.get('job') == 'Director':
            directors_list.append(person.get('name'))

    return directors_list

# keeping only directors
movies['crew'] = movies['crew'].apply(get_directors_names)

In [20]:
# convering overview into list so it will concatenate with other list
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
2033,10950,I Am Sam,"[Sam, has, the, mental, capacity, of, a, 7-yea...",[Drama],"[Sean Penn, Michelle Pfeiffer, Dianne Wiest, D...",[Jessie Nelson],"[foster parents, pro bono, new baby, social se..."


In [21]:
# combine all words so that will become one unique value
def combine_word(words):
    List = []
    for word in words:
        List.append(word.replace(" ",""))
    return List

In [22]:
# combining words for following columns only
movies['cast'] = movies['cast'].apply(combine_word)
movies['crew'] = movies['crew'].apply(combine_word)
movies['genres'] = movies['genres'].apply(combine_word)
movies['keywords'] = movies['keywords'].apply(combine_word)

# checking the dataset
movies.sample()

,movie_id,title,overview,genres,cast,crew,keywords
1888,9290,Crocodile Dundee in Los Angeles,"[After, settling, in, the, tiny, Australian, t...","[Adventure, Comedy]","[PaulHogan, LindaKozlowski, JereBurns, Jonatha...",[SimonWincer],"[crocodile, trafficjam, losangeles, stolenpain..."


In [23]:
# combining all required columns which can help to recommand user
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [24]:
# final data set
final_df = movies.drop(columns=['overview','genres','keywords','cast','crew'])
final_df.sample()

,movie_id,title,tags
4091,55604,Boom Town,"[McMasters, and, Sand, come, to, oil, towns, t..."


In [25]:
# now lets convert list into string format
final_df['tags'] = final_df['tags'].apply(lambda x: " ".join(x))
final_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [26]:
# converting all letter to lowercase
final_df['tags'] = final_df['tags'].apply(lambda x:x.lower())
final_df.sample()

,movie_id,title,tags
589,49017,Dracula Untold,"vlad tepes is a great hero, but when he learns..."


In [27]:
# Create an instance of the PorterStemmer
ps = PorterStemmer()

In [28]:
# Stemming words means converting created, creates or creative into one word create
def Stemming(text):
    List = []
    for word in text.split():
        List.append(ps.stem(word))
    return " ".join(List)

In [29]:
final_df['tags'] = final_df['tags'].apply(Stemming)
final_df.sample()

,movie_id,title,tags
266,2048,"I, Robot","in 2035, where robot are common-plac and abid ..."


In [30]:
# extracting common words
cv = CountVectorizer(max_features=5000,stop_words='english')


In [31]:
# creating vector for tags
vector = cv.fit_transform(final_df['tags']).toarray()

In [32]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
# checking the collected feature words
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [34]:
# for 4806 movies 2000 words
vector.shape

(4806, 5000)

In [35]:
# passing vector in cosine similarity
similarity = cosine_similarity(vector)

In [36]:
#checking similarity with first movie (should be 1)
similarity[0]

array([1.        , 0.075     , 0.08017837, ..., 0.04472136, 0.        ,
       0.        ])

In [37]:
# building recommendation function
def recommend(movie):
    index = final_df[final_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(final_df.iloc[i[0]].title)

In [38]:
#checking movies in dataset
final_df[final_df['title'] == 'Batman Begins']

,movie_id,title,tags
119,272,Batman Begins,"driven by tragedy, billionair bruce wayn dedic..."


In [39]:
# checking the recommendations
recommend('Batman Begins')

The Dark Knight
Batman
The Dark Knight Rises
Batman
Rockaway


In [40]:
#pick helps to download our model for building application
#pickle.dump(final_df.to_dict(),open('movie_list.pkl','wb'))
#pickle.dump(similarity,open('similarity.pkl','wb'))